# trying an XGBOOST Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error, r2_score
import matplotlib.pyplot as plt
%run preprocess_output.ipynb

In [2]:
columns_to_drop = []
data_train = get_train_data(columns_to_drop)

In [3]:
X = data_train.iloc[:, :-1]
Y = data_train['# arrivals']

res = train_test_split(X, Y, 
                       train_size=0.8,
                       test_size=0.2,
                       random_state=1)

X_train, X_test, Y_train, Y_test = res 

In [6]:
xgb_params = {
    'n_estimators': 30000,
    'learning_rate': 0.001,
    'subsample': 0.5,
    'alpha': .1,
    'max_depth': 15,
    'colsample_bytree' : .8,
    'min_child_weight': 1,
    'scale_pos_weight': 10,
    'objective': 'reg:squarederror',
    'device' : "cuda"
}

In [8]:
#Train the XGBoost model
xgb_model = XGBRegressor(**xgb_params)
xgb_model = xgb_model.fit(X_train, Y_train)
xgb_model.save_model("models/xgb_model_n30000_d15_lr1.json")

In [9]:
# xgb_model = XGBRegressor(**xgb_params)
# xgb_model.load_model("models/xgb_model_n20000_d15_lr1.json")
#Making predictions on the test set
predictions = xgb_model.predict(X_test)

# Calculate the mean squared error and R-squared score
rmse = root_mean_squared_error(Y_test, predictions)
r2 = r2_score(Y_test, predictions)

print("Root Mean Squared Error:", rmse)
print("R-squared Score:", r2)

/projectnb/cs640grp/students/samwu/Rnn-Forecast/.venv/lib64/python3.11/site-packages/xgboost/core.py:158: UserWarning: [10:33:27] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Root Mean Squared Error: 3.685607580435675
R-squared Score: 0.8727382713256637


# run on the test set

In [27]:
data_test = get_test_data(columns_to_drop)
dataset_test = data_test.copy()

220924
220924
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220925
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926
220926

In [73]:
X = dataset_test.iloc[:, :-1]
Y = dataset_test['# arrivals']
index_list = dataset_test.index.to_list()

#make predictions
xgb_model.load_model("models/xgb_model_n7200_d16.json")
#Making predictions on the test set
predictions = xgb_model.predict(X)
#predictions = [int(x) for x in predictions]
print(predictions)

[11.574767 12.898905 16.996445 ...  6.456597  7.007328  6.403601]


In [77]:
# Calculate the mean squared error and R-squared score
rmse = root_mean_squared_error(Y, predictions)
r2 = r2_score(Y, predictions)

print("Root Mean Squared Error:", rmse)
print("R-squared Score:", r2)

Root Mean Squared Error: 6.498682862147404
R-squared Score: -0.8214623601520168


In [59]:
predict_df = pd.DataFrame({"ID":index_list, "Prediction":list(predictions)}).set_index('ID')
predict_df

,Prediction
ID,
KATL_220924_0000,11.011949
KATL_220924_2345,12.184258
KATL_220925_0000,15.774439
KATL_220925_0015,11.368193
KATL_220925_0030,11.418930
...,...
KSEA_230820_0130,5.147779
KSEA_230820_0145,5.106507
KSEA_230820_0200,6.346481


In [63]:
submission_format_df = pd.read_csv("data/submission_format.csv")
post_process(submission_format_df,predict_df).to_csv("data/submission_XGB_n3200_d16.csv", index=False)